In [1]:
words = open('names.txt', 'r').read().splitlines()

In [2]:
import torch

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = { s:i+1 for i, s in enumerate(chars) }
stoi['.'] = 0
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [56]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [57]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1, ix2, ix3] += 1

In [58]:
N

tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0, 207, 190,  ...,  27, 173, 152],
         [  0, 169,   0,  ...,   0,   4,   0],
         ...,
         [  0,  57,   0,  ...,   1,  17,  11],
         [  0, 246,   0,  ...,   0,   0,   2],
         [  0, 456,   0,  ...,   0,  91,   1]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 40,   0,   5,  ...,   0,  20,  11],
         [ 36,  28,  20,  ...,   0,  12,   0],
         ...,
         [ 11,   5,   0,  ...,  17,   6,   3],
         [163, 389,  13,  ...,   0,  16,  40],
         [ 38, 123,   0,  ...,   0,  12,  22]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 46,   5,   5,  ...,   4,  31,   4],
         [  1,   8,   0,  ...,   0,   9,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [ 55,   4,   1,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 10,   0,   2,  ...,   0,  10,   0]

In [59]:
P = N.float()

In [62]:
P[26,26]

tensor([ 4., 13.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  8.,  0.,  0.,  5.,  4.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  7.,  0.])

In [39]:
P = (N+1).float()

In [40]:
P[0,5,13]

tensor(289.)

In [42]:
P[0,5].sum()

tensor(1558.)

In [43]:
289 / 1558

0.18549422336328628

In [24]:
P.sum(2, keepdim=True).shape

torch.Size([27, 27, 1])

In [32]:
P = P / P.sum(2, keepdim=True)
P.shape

torch.Size([27, 27, 27])

In [37]:
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [36]:
log_likelihood = 0.0
n = 0
for w in words[:3]:
# for w in ["andrejq"]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        prob = P[ix1, ix2, ix3]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}->{ch3}: {prob:.4f} {logprob:.4f}')

print (f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

.e->m: 0.1855 -1.6847
em->m: 0.1269 -2.0645
mm->a: 0.3744 -0.9825
ma->.: 0.0669 -2.7050
.o->l: 0.2494 -1.3887
ol->i: 0.1084 -2.2223
li->v: 0.0219 -3.8195
iv->i: 0.2669 -1.3209
vi->a: 0.1578 -1.8465
ia->.: 0.3657 -1.0060
.a->v: 0.0550 -2.9006
av->a: 0.1882 -1.6705
va->.: 0.1405 -1.9625
log_likelihood=tensor(-25.5742)
nll=tensor(25.5742)
1.9672455787658691


In [38]:
P[0,5,13]

tensor(0.1855)

In [50]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(P[0,0], num_samples=1, replacement=True, generator=g).item()
ix

10

In [55]:
P[0,5]

tensor([  1.,  24.,  16.,   5.,  81.,  10.,  11.,   7.,  10.,  55.,   4.,   9.,
        489., 289.,  61.,   5.,   9.,   1.,  91.,  94.,  21.,  22., 155.,   5.,
          5.,  25.,  53.])